In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-07"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11682,2024-04-07,Espanha Acb,07:30,Manresa,Unicaja,2.53,1.53,167.5,1.84,1.92,3.5,1.98,1.29,06592I95,0.395257,0.653595,0.543478,0.520833,0.048852,0.4,0.4,NaN,NaN,174.940,39.416762,0.225316,142.344,35.768333,0.251281,160.486,137.314,138.84,122.40,0.0,0.0,0.0,0.0,0.348328,0.030090,0.298412,-1.85,-0.370,-4.135135,0.595653,0.6,0.004347,1.50,0.300,1.766667,0.598873,0.9,0.301127
11683,2024-04-07,Espanha Acb,12:00,Valencia,Baskonia,1.58,2.40,165.5,1.85,1.93,-4.5,2.00,3.40,Q16MDeta,0.632911,0.416667,0.540541,0.518135,0.049578,0.6,0.6,NaN,NaN,169.642,79.785705,0.470318,149.946,52.691629,0.351404,108.930,162.414,122.01,130.40,0.0,0.0,0.0,0.0,0.291371,0.029930,0.366648,-2.34,-0.468,-1.239316,0.719973,0.6,-0.119973,-1.94,-0.388,-3.608247,0.000000,0.0,0.000000
11684,2024-04-07,Espanha Acb,13:30,Barcelona,Real Madrid,1.73,2.12,167.5,1.86,1.94,-2.5,1.95,2.42,4h5QCFe5,0.578035,0.471698,0.537634,0.515464,0.049733,0.4,0.6,NaN,NaN,119.816,27.294778,0.227806,107.882,20.344356,0.188580,100.408,120.804,96.52,141.81,0.0,0.0,0.0,0.0,0.143258,0.029773,0.152101,-1.55,-0.310,-2.354839,0.000000,0.0,0.000000,-0.43,-0.086,-13.023256,0.751918,0.8,0.048082
11685,2024-04-07,Eua Nba,16:30,Los Angeles Clippers,Cleveland Cavaliers,1.52,2.65,218.5,1.80,2.01,-4.5,1.93,3.22,AeNlloXf,0.657895,0.377358,0.555556,0.497512,0.035253,0.2,0.2,NaN,NaN,195.854,34.760607,0.177482,195.210,26.444989,0.135469,161.710,178.620,204.75,219.22,0.0,0.0,0.0,0.0,0.383228,0.077949,0.354240,-3.93,-0.786,-0.661578,0.724256,0.6,-0.124256,-3.71,-0.742,-2.223720,0.582235,0.5,-0.082235
11686,2024-04-07,Eua Nba,18:00,Indiana Pacers,Miami Heat,1.84,2.03,228.5,1.81,1.95,-2.5,1.99,2.20,lnMhm5n1,0.543478,0.492611,0.552486,0.512821,0.036089,0.2,0.6,NaN,NaN,213.108,64.660366,0.303416,278.970,81.652854,0.292694,197.124,283.758,180.18,229.67,0.0,0.0,0.0,0.0,0.069432,0.052657,0.070879,0.05,0.010,84.000000,0.576446,0.7,0.123554,4.90,0.980,1.051020,0.443165,0.7,0.256835
11687,2024-04-07,Itália Liga A,11:30,Varese,Basket Napoli,2.22,1.68,175.5,1.82,1.92,1.5,2.03,1.53,hhWuO2US,0.450450,0.595238,0.549451,0.520833,0.045689,0.6,0.8,NaN,NaN,217.304,55.501337,0.255409,261.306,117.103398,0.448147,218.260,271.810,270.48,424.20,0.0,0.0,0.0,0.0,0.195814,0.037813,0.198626,-1.43,-0.286,-4.265734,0.486442,0.4,-0.086442,-0.17,-0.034,-20.000000,0.452567,0.4,-0.052567
11688,2024-04-07,Itália Liga A,13:00,Brindisi,Treviso,1.69,2.19,160.5,1.85,1.93,-3.5,2.02,2.51,bVWyPrFM,0.591716,0.456621,0.540541,0.518135,0.048337,0.4,0.6,NaN,NaN,253.094,215.085590,0.849825,227.876,49.504675,0.217244,151.164,239.504,126.00,212.52,0.0,0.0,0.0,0.0,0.182244,0.029930,0.152972,-1.55,-0.310,-2.225806,0.455780,0.5,0.044220,3.21,0.642,1.853583,0.000000,0.0,0.000000
11689,2024-04-07,Itália Liga A,13:15,Venezia,Virtus Bologna,2.24,1.66,162.5,1.87,1.92,2.5,1.92,1.42,KdBtZseK,0.446429,0.602410,0.534759,0.520833,0.048838,0.6,0.0,NaN,NaN,157.028,28.463814,0.181266,128.648,20.076517,0.156058,137.460,119.978,185.64,156.60,0.0,0.0,0.0,0.0,0.210319,0.018657,0.211709,1.68,0.336,3.690476,0.000000,0.0,0.000000,0.67,0.134,4.925373,0.000000,0.0,0.000000
11690,2024-04-07,Itália Liga A,14:30,Pistoia,Reggiana,1.84,1.97,156.5,1.86,1.90,-1.5,1.95,2.10,f1I7JMxp,0.543478,0.507614,0.537634,0.526316,0.051092,0.2,0.2,NaN,NaN,260.694,133.039235,0.510327,181.832,71.769350,0.394701,161.986,174.056,126.99,154.86,0.0,0.0,0.0,0.0,0.048254,0.015045,0.052378,0.52,0.104,8.076923,0.482135,0.6,0.117865,-3.22,-0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,12:00,Espanha Acb,Valencia,Baskonia,165.5,1.85,1.0000,Over
1,13:30,Espanha Acb,Barcelona,Real Madrid,167.5,1.86,1.0000,Over
2,16:30,Eua Nba,Los Angeles Clippers,Cleveland Cavaliers,218.5,1.80,1.0000,Over
3,18:00,Eua Nba,Indiana Pacers,Miami Heat,228.5,1.81,1.0000,Over
4,13:00,Itália Liga A,Brindisi,Treviso,160.5,1.85,1.0000,Over
5,14:30,Itália Liga A,Pistoia,Reggiana,156.5,1.86,1.0000,Over
6,10:00,Alemanha Pro A,Bayreuth,Bremerhaven,167.5,1.85,0.9993,Over
7,01:00,Austrália Nbl1 Sul,Ballarat,Geelong,180.5,1.83,0.9981,Over
8,04:05,Japão B.League,Brave Thunders,Hiroshima D.,154.5,1.85,1.0000,Over
9,10:45,Libano Divisão 1,Champville,Homenetmen Beirut,171.5,1.85,1.0000,Over
